In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *

# data

In [ ]:
# context = Context('amended-1-no_noise', debug=0, dep_var=dep_var2, mulr=1, gpu_start=9)
context = Context('amended-3-7', debug=0, dep_var=dep_var2, mulr=7, gpu_start=9)
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

In [ ]:
# read context and append 
geo_raw = pd.read_csv('/home/wenxiang/datasets/tbm/geo_csv/geo2.1.csv', index_col=0)
geo_cols = ['起始桩号', '最后桩号', '实际开挖围岩类别', '洞室部位主要岩性']
geo = geo_raw.loc[:,geo_cols]
# clean nan
geo = geo.loc[geo.iloc[:,-1].notna()]
zh = context.cyc_cont.iloc[:,-5]
cat_names = geo.columns.tolist()

idx_dep_var3 = []
missing_idx = -1 #geo.shape[0] # last item + 1
for z in list(zh):
    is_within = ((z - geo.iloc[:,0]) * (z - geo.iloc[:,1])) < 0
    if is_within.any():
        idx_dep_var3.append(np.argmax(is_within.values))
    else:
        idx_dep_var3.append(missing_idx)

dep_var3 = []
for i in idx_dep_var3:
    dep_var3.append(geo.iloc[i,2:].tolist() if i != -1 else ['oor', 'oor'])

cycles = pd.concat([context.cyc_cont, pd.DataFrame(dep_var3, columns=['围岩等级', '岩性'])], axis=1)
cat_names = cycles.columns[-2:]
cont_names = cycles.columns[:-2]
cf = Categorify(cat_names, cont_names)
cf.apply_train(cycles)

fn_feather = Path(context.fn_feather)
new_fn_feather = fn_feather.parent / (fn_feather.stem + '-cat' + fn_feather.suffix)
cycles.to_feather(new_fn_feather)

# run

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *

In [ ]:
n_cont = 2
n_cat = [9, 6]
cat_names = ['围岩等级', '岩性']

# context = Context('amended-1-no_noise', debug=0, dep_var=dep_var2, mulr=1, gpu_start=9)
context = Context('amended-3-7-cat', debug=0, dep_var=dep_var2 + cat_names, mulr=7, gpu_start=9)
cycles = context.cyc_cont

# categorify
cat_columns = []
for cat_name in cat_names:
    cat_columns.append(cycles.loc[:,cat_name])
    cycles.loc[:,cat_name] = cycles.loc[:,cat_name].cat.codes.values
    
databunch = MultiDeptTabularDataBunch.from_df('tmp', cycles, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

# model

In [ ]:
hyper_params = {
    'use_extra_x': 1,
    'n_hidden': 1000,
    'n_layers': 3,
    'hidden_p': 0.3,
    'input_p' : 0.4,
    'weight_p': 0.5,
    'drops': [0.4] * 4,
    'alpha': 2.,
    'beta': 1.,
    'clip': 1,
    'loss_func': DualLoss(n_cont, n_cat, 0.5)
}

hyper_params['layers'] = [3*(context.n_cont) + int(hyper_params['use_extra_x']) * 3, 1000, 17]
learner = get_new_model(context, databunch, hyper_params)

learner.fit(20, lr=1e-1)

In [ ]:
hyper_params['layers']

In [ ]:
hyper_params = {
    'use_extra_x': 1,
    'n_hidden': 1000,
    'n_layers': 3,
    'hidden_p': 0.3,
    'input_p' : 0.4,
    'weight_p': 0.5,
    'drops': [0.4] * 4,
    'alpha': 2.,
    'beta': 1.,
    'clip': 1,
    'loss_func': DualLoss(n_cont, n_cat, 1e-2)
}

hyper_params['layers'] = [3*(context.n_cont) + int(hyper_params['use_extra_x']) * 3, 1000, 17]
learner = get_new_model(context, databunch, hyper_params)

learner.fit(20, lr=1e-1)

In [ ]:
learner.fit(40, lr=1e-2)

In [ ]:
xb_valid, yb_valid = ni(databunch.valid_dl)
yb_valid = to_np(yb_valid)

In [ ]:
model = learner.model
model.eval()
model.reset()
pb_valid = model(*xb_valid)[0]

# pb = denormalize(pb[0], *context.stat_y)
# pb = pb[0]
# pb = to_np(pb)
# pb_valid = pb_valid[0]
# pb_valid = to_np(pb_valid)

In [ ]:
p_cont, p_cat = pb_valid[:,:n_cont], pb_valid[:,n_cont:]

In [ ]:
cat_idx_zip = [0] + hyper_params['loss_func'].cat_idx.tolist()
p_cat_tasks = []
for i,j in zip(cat_idx_zip[:-1], cat_idx_zip[1:]):
    p_cat_tasks.append(to_np(p_cat[:,i:j]))

In [ ]:
p1, p2 = [o.argmax(1) for o in p_cat_tasks]

In [ ]:
yb_valid[:,-2:]

In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
class_names = cat_columns[0]

In [ ]:
class_names = cat_columns[0].cat.categories

In [ ]:
y_test = yb_valid[:,-2].astype(np.int)
y_pred = p1
plot_confusion_matrix(y_test, y_pred, classes=class_names,
                      title='Confusion matrix, without normalization')

In [ ]:
y_test = yb_valid[:,-2].astype(np.int)
y_pred = p1
plot_confusion_matrix(y_test, y_pred, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')